# Marlowe Lang


Some basic usage-example for marlowe_lang.
There is also an interactive example published at [runkit.com](https://runkit.com/olofblomqvist/marlowe-dsl-example)

In [1]:
let m = require('marlowe_lang')

marlowe_lang utils initialized.


#### Working with JSON

In [2]:
// Converting Marlowe DSL to JSON
let json_encoded_contract = m.marlowe_to_json_with_variables('When [] (TimeParam "var1") Close',"var1=123") // or just marlowe_to_json if you dont need variables
json_encoded_contract
// if you just want to populate variables without converting to json, you can do:
// m.parse_marlowe_with_variables(contract,"Price=1")

{
  "when": [],
  "timeout_continuation": "close",
  "timeout": 123
}


In [3]:
m.decode_marlowe_dsl_from_json(json_encoded_contract)

When [  ] 123 Close


#### Using the state machine

#### Viewing current contract state
**ie. next acceptable input actions**

In [4]:
let m = require('marlowe_lang')

// Examine some contract state
let contract = `When
    [Case
        (Deposit
            (Role "Seller")
            (Role "Buyer")
            (Token "" "")
            (ConstantParam "Price")
        )
        Close ]
    3487663680000 Close 
`
// Populate the variables
let core_dsl = m.parse_marlowe_with_variables(contract,'Price=1000000');

// Create state machine and process until it closes or requires input
let machine = new m.WASMMarloweStateMachine(core_dsl,"");
machine.process();

// Read current state and print to console
let current_state = machine.machine_state_json();
current_state

{
  "WaitingForInput": {
    "expected": [
      {
        "Deposit": {
          "who_is_expected_to_pay": {
            "role_token": "Buyer"
          },
          "expected_asset_type": {
            "token_name": "",
            "currency_symbol": ""
          },
          "expected_amount": 1000000,
          "expected_target_account": {
            "role_token": "Seller"
          },
          "continuation": "close"
        }
      }
    ],
    "timeout": 3487663680000
  }
}


#### Parameters

Marlowe has support for using variables, but those need to be set prior to initializing a contract so that it can run.
You can inspect the parameters for a contract using "get_input_params_for_contract", and set them using "parse_marlowe_with_variables"

In [5]:
let param_list = m.get_input_params_for_contract(contract)
let marlowe_core_dsl = m.parse_marlowe_with_variables(contract,"Price=666")

console.log("The contract has the following variables",param_list)
console.log("After populating the 'Price' variable, the contract looks like this (marlowe core dsl)",marlowe_core_dsl)

The contract has the following variables [ 'CONST_PARAM:Price' ]
After populating the 'Price' variable, the contract looks like this (marlowe core dsl) When [ (Case (Deposit (Role "Seller") (Role "Buyer") (Token "" "") (Constant 666)) Close) ] 3487663680000 Close


#### Working with CBORHEX

In [6]:
// Some example datum from here: https://preprod.cexplorer.io/tx/0b4fbb9e3ab2ada2249ab3b8b8f7520a833c224c19420f1fecef22601f9d6218
let example_datum_cbor_hex = "d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd8799f581c9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe51446a65645f746573744d6963726f555344ffd87a9f1a3b9aca00ffffd87a9fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd87a9fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffffd8799f581c9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe51446a65645f746573744d6963726f555344ffd87a9f1a3b9aca00ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd8799f581c9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe51446a65645f746573744d6963726f555344ffd87f9fd87e9fd87a9f1a000186a0ffd87a9f1a3b9aca00ffffd87a9f1a000f4240ffffffd87a9fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd87a9fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffffd8799f581c9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe51446a65645f746573744d6963726f555344ffd87f9fd87e9fd87a9f1a000186a0ffd87a9f1a3b9aca00ffffd87a9f1a000f4240ffffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd8799f581c9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe51446a65645f746573744d6963726f555344ffd87a9f1a3b9aca00ffffd87a9fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffd87a9fd8799fd87980d8799fd8799f581c8b4e5029a73fd4155dfdcb5013e28db2fafb3e1ab487275f4aa50903ffd8799fd8799fd8799f581ccce6bfa02342d29adce20d09d3824463bfb59b0753d7d1d8514b376cffffffffffffd8799f581c9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe51446a65645f746573744d6963726f555344ffd87a9f1a3b9aca00ffd87980ffffff1b000001941f297c00d87980ffffffff1b000001941f297c00d87980ffffffff1b0000018cc251f400d87980ffff"

// example from here: https://cardanoscan.io/transaction/f3a397d2f58f2c30a8034235659c5688b0c5d308581dbd7ce4470049b4aebd66?tab=contracts
let example_redeemer_cbor_hex = "9fd8799fd8799fd8799fd87a80d8799fd8799f581c1fdc22c9b2339e644d229335129b35e65161839660636675b29aa5a8ffd8799fd8799fd8799f581c43b85595ee70ca74e98583872933bec6c67a99982ca2f25547a89b4dffffffffffd8799fd87a80d8799fd8799f581c1fdc22c9b2339e644d229335129b35e65161839660636675b29aa5a8ffd8799fd8799fd8799f581c43b85595ee70ca74e98583872933bec6c67a99982ca2f25547a89b4dffffffffffd8799f581c85556ab05acc925edff2af02ef819a8b4903bfb5d5e100a7c95ab9084d4d61726c6f77654c6973626f6eff01ffffff"


#### Decoding datums

In [7]:
m.decode_cborhex_marlowe_plutus_datum(example_datum_cbor_hex)


{
  "marlowe_params": "",
  "state": {
    "accounts": [
      [
        [
          {
            "address": "addr_test1qz95u5pf5ulag92alh94qylz3ke047e7r26gwf6lf2jsjq7vu6l6qg6z62ddecsdp8fcy3rrh76ekp6n6lgas52txakq3eft7r"
          },
          {
            "token_name": "",
            "currency_symbol": ""
          }
        ],
        2000000
      ]
    ],
    "choices": [],
    "boundValues": [],
    "minTime": 0
  },
  "contract": {
    "when": [
      {
        "then": {
          "token": {
            "token_name": "Djed_testMicroUSD",
            "currency_symbol": "9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe"
          },
          "to": {
            "party": {
              "address": "addr_test1qz95u5pf5ulag92alh94qylz3ke047e7r26gwf6lf2jsjq7vu6l6qg6z62ddecsdp8fcy3rrh76ekp6n6lgas52txakq3eft7r"
            }
          },
          "then": {
            "when": [
              {
                "then": {
                  "token": {
                    "token_n

#### Decoding redeemer/inputs

In [8]:

m.decode_marlowe_input_cbor_hex(example_redeemer_cbor_hex)

{
  "Ok": [
    {
      "input_from_party": {
        "address": "addr1qy0acgkfkgeeuezdy2fn2y5mxhn9zcvrjesxxen4k2d2t2zrhp2etmnsef6wnpvrsu5n80kxceafnxpv5te923agndxshwgt4u"
      },
      "of_tokens": {
        "token_name": "MarloweLisbon",
        "currency_symbol": "85556ab05acc925edff2af02ef819a8b4903bfb5d5e100a7c95ab908"
      },
      "into_account": {
        "address": "addr1qy0acgkfkgeeuezdy2fn2y5mxhn9zcvrjesxxen4k2d2t2zrhp2etmnsef6wnpvrsu5n80kxceafnxpv5te923agndxshwgt4u"
      },
      "that_deposits": 1
    }
  ]
}


#### Try-decode

If you don't know if the cbor-hex is a valid redeemer or datum, you can use the "decode_marlowe_data_or_redeemer" method
which will attempt to figure out the correct way to decode it

In [9]:

m.decode_marlowe_data_or_redeemer(example_redeemer_cbor_hex)

Decoded redeemer from cbor hex:

 (Deposit (Address "addr1qy0acgkfkgeeuezdy2fn2y5mxhn9zcvrjesxxen4k2d2t2zrhp2etmnsef6wnpvrsu5n80kxceafnxpv5te923agndxshwgt4u") (Address "addr1qy0acgkfkgeeuezdy2fn2y5mxhn9zcvrjesxxen4k2d2t2zrhp2etmnsef6wnpvrsu5n80kxceafnxpv5te923agndxshwgt4u") (Token "85556ab05acc925edff2af02ef819a8b4903bfb5d5e100a7c95ab908" "MarloweLisbon") 1)


#### Encoding contracts to cbor-hex

Todo: This is currently not exposed in the WASM builds but only using the Rust crate.
